# 1. Ingestione e Preparazione dei Dati

**Obiettivo:** Caricare i dati grezzi, pulirli e prepararli per le analisi successive. Questa è la base di tutto il progetto, quindi è fondamentale farla bene.

1.  **Setup dell'Ambiente:** Configuro PySpark e importo le librerie che mi servono.
2.  **Caricamento Dati:** Scarico automaticamente il dataset da Kaggle e lo carico in uno DataFrame Spark.
3.  **Pulizia e Standardizzazione:** Applico le funzioni per standardizzare i nomi delle colonne, correggere i tipi di dato e gestire i valori mancanti.
4.  **Salvataggio:** Salvo il DataFrame pulito in formato Parquet, che è ottimo per lavorare con Spark.

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.config.spark_config import SPARK_CONFIG
from src.data_ingestion.download_data import download_nba_dataset
from src.data_processing.cleaning import (correct_data_types, 
                                          handle_missing_values, 
                                          standardize_column_names)
from src.utils.helpers import get_spark_session, save_dataframe

project_root = os.path.abspath(os.path.join('..'))
raw_data_dir = os.path.join(project_root, "data", "raw")
processed_data_dir = os.path.join(project_root, "data", "processed")
csv_file_path = os.path.join(raw_data_dir, "Player Totals.csv")
output_path = os.path.join(processed_data_dir, "players_cleaned.parquet")

spark = get_spark_session(
    app_name="NBA_Data_Ingestion_Preparation",
    driver_memory=SPARK_CONFIG["driver_memory"]
)

### Fase 1: Caricamento Dati con Download Automatico

Adesso, controllo se ho già il dataset. Se non c'è, lo scarico da Kaggle. Poi lo carico in Spark e do una prima occhiata per capire com'è fatto.

In [ ]:
if not os.path.exists(csv_file_path):
    print(f"Dataset non trovato. Lo scarico in '{raw_data_dir}'...")
    download_nba_dataset(raw_data_dir)
else:
    print(f"Dataset già presente in '{csv_file_path}'. Salto il download.")

raw_df = spark.read.csv(csv_file_path, header=True, inferSchema=False)

print(f"\nNumero di righe iniziali: {raw_df.count()}")
print("Schema iniziale del DataFrame:")
raw_df.printSchema()
raw_df.show(5, truncate=False)

#### Interpretazione dell'output

Ok, vedo che ho circa 27.000 righe. Come mi aspettavo, tutte le colonne sono state caricate come `string`. Dovrò sistemarle.

La tabella di esempio mi conferma che ci sono nomi di colonna strani (es. `3P%`) e valori `NA` da gestire. La pulizia sarà necessaria.

### Fase 2: Pulizia e Preparazione dei Dati

Questa è la fase più importante di pre-processing. Applicherò le funzioni in sequenza:
1.  **`standardize_column_names`**: Rendo i nomi delle colonne puliti e facili da usare.
2.  **`correct_data_types`**: Converto le colonne numeriche da `string` a `double`.
3.  **`handle_missing_values`**: Filtro i dati per tenere solo le stagioni dall'era del tiro da tre e i giocatori con un numero minimo di partite giocate.

In [ ]:
df_std_names = standardize_column_names(raw_df)
df_typed = correct_data_types(df_std_names)
df_cleaned = handle_missing_values(
    df_typed, 
    min_games_threshold=SPARK_CONFIG["min_games_threshold"]
)

print(f"Numero di righe dopo la pulizia: {df_cleaned.count()}")

df_cleaned.select("player", "season", "pts", "mp", "g").show(10)

#### Interpretazione dell'output

Il numero di righe si è ridotto parecchio. Questo è un buon segno, significa che ho tolto i dati che non erano rilevanti per la mia analisi (stagioni troppo vecchie, giocatori con poche partite).

Ora le colonne `pts`, `mp` e `g` sono numeriche e pronte per i calcoli. I dati sembrano molto più robusti.

### Fase 3: Salvataggio dei Dati Processati

Ora che il DataFrame è pulito, lo salvo in formato Parquet. Così sarà veloce da caricare nei prossimi notebook e manterrà i tipi di dato corretti.

In [ ]:
save_dataframe(df_cleaned, output_path)

print(f"DataFrame pulito salvato con successo in '{output_path}'.")

### Conclusione e Prossimi Passi

Ho completato con successo la prima fase: ho caricato, pulito e preparato i dati. Il risultato è il file `players_cleaned.parquet`, che sarà il punto di partenza per la prossima fase.

Nel prossimo notebook, `02_exploratory_data_analysis.ipynb`, userò questo dataset pulito per fare un'analisi esplorativa e cercare le prime intuizioni.

In [ ]:
spark.stop()